In [ ]:
!pip install mlflow -q

In [ ]:
from enum import Enum

from tensorflow.keras.applications.mobilenet_v2 import (
    preprocess_input as preprocess_input_mobilenet,
)
from tensorflow.keras.applications import ResNet50, MobileNetV2
from tensorflow.keras.applications.resnet import (
    preprocess_input as preprocess_input_resnet,
)
from tensorflow.keras.optimizers import SGD, Adam, RMSprop


class OptimizerType(str, Enum):
    ADAM = Adam
    SGD = SGD
    RMSPROP = RMSprop


class CNNType(str, Enum):
    RESNET = ResNet50
    MOBILENET = MobileNetV2


optimizer_map = {
    "adam": OptimizerType.ADAM,
    "sgd": OptimizerType.SGD,
    "rmsprop": OptimizerType.RMSPROP,
}
cnn_map = {"resnet50": CNNType.RESNET, "mobilenetv2": CNNType.MOBILENET}
preprocess_img_map = {
    "resnet50": preprocess_input_resnet,
    "mobilenetv2": preprocess_input_mobilenet,
}

# ------ HYPERPARAMETERS (Default Values) ------
BATCH_SIZE = 12
EPOCHS = 10
LEARNING_RATE = 0.001
OPTIMIZER_str = "adam"
CNN_str = "resnet50"

# ------ RUN_ID ------
PREFECT_RUN_ID = "Default"
EXPERIMENT_NAME = "Default"
RUN_NAME = "Avocado_Ripening_Model"

SyntaxError: ':' expected after dictionary key (2404400698.py, line 32)

In [ ]:
OPTIMIZER = optimizer_map[OPTIMIZER_str]
CNN = cnn_map[CNN_str]
print(f"OPTIMIZER: {OPTIMIZER}")
print(f"CNN: {CNN}")
print(f"BATCH_SIZE: {BATCH_SIZE}")
print(f"EPOCHS: {EPOCHS}")
print(f"LEARNING_RATE: {LEARNING_RATE}")

In [ ]:
import mlflow.sklearn
import numpy as np
import requests
from sklearn.svm import LinearSVC

import mlflow

MLFLOW_URI = "https://mlflow.lepcodes.com"


def test_mlflow_connection():
    print(f"📡 Probando conexión a {MLFLOW_URI}...")
    try:
        # 1. Ping rápido al servidor
        response = requests.get(f"{MLFLOW_URI}/health")
        if response.status_code != 200 and response.status_code != 404:
            print(
                f"El servidor respondio con codigo {response.status_code}, pero parece vivo."
            )
        else:
            print("Conexion establecida con el servidor MLflow.")

        mlflow.set_tracking_uri(MLFLOW_URI)
        print(f"Tracking URI configurado: {mlflow.get_tracking_uri()}")

    except Exception as e:
        print("ERROR CRITICO: No se puede conectar a MLflow.")
        print(f"Causa: {e}")
        print("Revisa que tu contenedor 'mlflow' este corriendo (docker ps).")
        return False
    return True


def run_dummy_experiment():
    experiment_name = EXPERIMENT_NAME

    # 3. Crear o Establecer Experimento (Idempotente)
    try:
        mlflow.set_experiment(experiment_name)
    except:
        try:
            mlflow.create_experiment(experiment_name)
            mlflow.set_experiment(experiment_name)
        except Exception as e:
            print(f"Error configurando experimento: {e}")
            return

    print(f"Usando experimento: {experiment_name}")

    # 4. Iniciar Run
    with mlflow.start_run(run_name=RUN_NAME) as run:
        print(f"Run {RUN_NAME} iniciado con ID: {run.info.run_id}")

        # Set prefect run id as tag
        print(f"Prefect Run ID: {PREFECT_RUN_ID}")
        mlflow.set_tag("prefect_run_id", PREFECT_RUN_ID)

        # Loguear Parametros Dummy
        mlflow.log_param("tipo_prueba", "conectividad")
        mlflow.log_param("autor", "luispreciado99")

        # Generar datos falsos y modelo
        X = np.array([0, 0, 1, 1, 1, 0]).reshape(-1, 1)
        y = np.array([0, 0, 1, 1, 1, 0])
        model = LinearSVC()
        model.fit(X, y)

        # Loguear Métrica
        score = model.score(X, y)
        mlflow.log_metric("accuracy_test", score)
        print(f"Métricas logueadas (Accuracy: {score})")

        # Loguear Modelo
        print("Subiendo modelo (artefacto)...")
        mlflow.sklearn.log_model(mdel, "avocado-model")

        print("¡EXITO TOTAL!")
        print(
            f"Revisa tu UI en: {MLFLOW_URI}/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}"
        )


# --- EJECUCIÓN ---
if test_mlflow_connection():
    run_dummy_experiment()

